## Return He-3 concentrations
<br>
  
#### Syntax
`CalcPars = HeLoopFxn(RunPars, CosmoPars, HeliumPars, CalcPars)` <br>
#### Input 
`RunPars` : dictionary of parameters relevant for the whole model run <br>
`CosmoPars` : dictionary of parameters relevant for CRN production/decay <br>
`HeliumPars` : dictionary of parameters relevant for He-3 production/diffusion <br>
`CalcPars` : dictionary of parameters with return CRN + He-3 calculations <br>
#### Variables Used
##### Run Pars
`RockfallMatrix` : matrix of rockfall depths size s * total_time <br>
`total_time` : total years of the run (yr) <br>
`s` : number of scenarios <br> 
`dz` : depth node spacing (cm) <br> 
`Hez` : Helium node depths (cm) <br>
`nx` : number of He grain nodes <br>
`shell_mass` : vector size nx with mass of qtz grain shells for He3 diffusion (g) <br>
#### Output
##### Calc Pars
`He Surf Matrix` : matrix size Hez with concentration of 3He at depth for given grain size and density <br>
#### Notes
**Date of Creation:** 7. Juli 2021 <br>
**Author:** Donovan Dennis <br>
**Update:** <br>

In [5]:
def HeLoopFxn(RunPars, CosmoPars, HeliumPars, CalcPars):
    RockfallMatrix = RunPars['RockfallMatrix']
    total_time = RunPars['total_time']
    s = RunPars['scenarios']
    dz = RunPars['dz']
    
    Hez = HeliumPars['Hez']
    nx = HeliumPars['nx']
    shell_mass = HeliumPars['shell_mass']
    
    HeliumPars = ReturnTempsHe3(RunPars, HeliumPars)
    HeSurfMatrix = np.empty((total_time,s))
    
    for i in range(s):
        conc3He = [[0]*nx] * len(Hez)
        HeSurf = np.empty((total_time,))
            
        for j in range(total_time):
            erode_z = RockfallMatrix[i,j]
            conc3He = HeErodeFxn(conc3He, erode_z, CosmoPars, HeliumPars)
            HeSurf[j] = np.average([np.average(conc3He[i], weights = shell_mass) for i in range(int(6))])
        
        HeSurfMatrix[:,i] = HeSurf
    
    CalcPars['conc3He']
    CalcPars['He Surf Matrix'] = HeSurfMatrix
    
    return CalcPars